In [2]:
from collections import defaultdict
from elasticsearch import Elasticsearch, NotFoundError
from elasticsearch_dsl import Search
import numpy as np
import ssl
from elasticsearch.connection import create_ssl_context
from elasticsearch_dsl import Q
from graphene import ObjectType, String, Int, ID, Float, List, Schema
import urllib3

from onai.ml.peers.ranker import Ranker

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


class Company(ObjectType):
    id = ID()
    name = String()
    country = String()
    sector = String()
    revenue = Float()
    ebitda =Float()
    ebitda_marg = Float()
    num_employees = Int()
    score = Float()
    region = String()
    description = String()
    company_type = String()
    cosine_similarity = Float()


public_labels = {
'1b3872ba-8913-517f-a253-7e62a525cf55':['f702fd12-611d-51ba-b61d-c95d505516f5', 'e449976b-580d-546a-aeb2-475a36c48d5c', '91a4576f-d032-5e06-a8a9-5753e836a2d1', '2978b28a-96d5-5c2e-b212-39893c36ed0a'],
'7689e398-0970-5c18-ba93-e0b14013ab30':['e5e55e0c-98f2-57e6-b252-2acd32197a33', 'dbd74ef8-7e63-587c-a0db-02846764cebd', '29f2b914-f655-5a0e-8642-252840119f94'],
'cb997a23-552a-52b9-acf5-bcdce2d1eea1':['f216296d-2161-54dc-a567-72a3043bf55b', '7ee2b159-1c7b-5867-92dd-8323be244577', 'b21e9a87-c5a6-5906-9d60-518f12f4a015'],
'e62877cd-15cd-561d-a77e-8a5bccf9873c':['ea39af2b-a76b-5df5-bcd1-c785cabedfe0', '9f8a1e48-a744-50cb-9b0d-87febeddb95b'],
'29f2b914-f655-5a0e-8642-252840119f94':['ea39af2b-a76b-5df5-bcd1-c785cabedfe0', '44b37e05-b67e-5ab4-bc3c-6d9e090ac13d', '07e02755-93b9-51d4-ab1f-67bc3ac4593c'],
'c655ffcb-391e-57af-9d57-28586381be92':['74116d9c-f19e-5089-b8d0-894befb4309b', 'bc68fb91-85c2-5650-bca5-dfbd0a1dd1ee', '7689e398-0970-5c18-ba93-e0b14013ab30'],
'b01d22f1-2b08-5a1b-937e-bd29be833832':['6d4e8b59-6790-5bad-9578-dfc5e23d40e6', 'a3c20fd1-df51-5c6b-9cbc-f1e5dd2ac129', 'aae8652a-0670-5643-978b-8e596552b4ff', '5147c270-2d82-5f2f-9081-62966a777e63', '7d48041c-b5b4-531b-9570-7deff9cc6c2e', 'a22a6d38-8932-5bb2-859b-cbb8ad9cf2a9', 'bffaf4bf-d404-5af9-bd7a-08110d90d39e', '8c0231ed-d771-5821-933e-15a0be075d88', 'a659d2df-a86d-595a-8420-1d42f82d66e4', 'd96d1bd4-fa15-5267-a659-42f973d5603b'],
'6a99e295-db08-5598-9746-10848bd2871e':['fc7226d1-910a-5010-bdb9-2d7865355965', 'd3189f4b-7dd2-5751-a8d2-a2121b0b4790', '7f943915-83be-505f-b065-bd45ccba65e2'],
'eb6a1519-3559-5797-ac35-d31505d66961':['6deb9c74-e7ed-551a-b88f-a107678b22c1', 'aa0ef5d1-7a0e-518d-b93d-1834db5bae48', 'ddc78420-8f1e-5656-a2fa-c967f278aad1', '65c843d5-e8b8-537f-a6c4-3ccb301d09e5', 'ba3be4f0-592d-5d72-a96b-5c461c871c9b', '522170f9-6489-52b8-bba6-ccb4ea4663ee', '005d3b86-14de-58a3-86ac-16ceff561782'],
'94a8555e-9f82-5657-9f32-f44c1844e371':['c7c22dd5-005b-56f1-8bd3-3420cc515895', '7fc2ed34-67db-5aed-bd9b-4a93d7981384', '8a586930-f14b-5414-902d-9c011a035e22', 'aab5d77c-76a1-5a64-ba9e-9db25d7126a0', 'f83f09b6-7a21-50c4-9cf9-730278fe565e'],
'4dfb3296-c5ab-5d27-bcaa-71a67990dc0f':['5db27606-2ba6-5599-aab1-754449cbf30f', '8b881676-4524-54fb-8d04-e9d80a70adc6'],
'3e71b765-5a7c-54b0-8687-fbb479e5ce91':['cc6a39ed-58d8-55d0-909f-ba1fba65159c', '42f7f27d-ed06-58e3-88c4-0d8e347991d7', '40cb3da1-7886-575f-b208-dd5e3d2e50ec', '648c94fc-5575-5e26-b284-66332d12d98d', 'db6e81fa-d08d-5597-bc1c-f3e97c72e3f3', 'c9fddd7f-7bd2-564a-acab-a776d09988cb'],
'4b90ea7b-dd01-5114-b2b1-e44f8b0f496c':['ff551276-08ea-5656-a96e-8f9b7c620de7', 'bade0809-2003-5dbb-bf6d-1aefe7347bd5', '2c437ad1-51ed-52bc-ac6f-b9d549a81370', '2cdc6f88-488e-5a6a-92e0-7d0c3a6f9c6b', 'a06b3a79-9328-5cc5-b8d1-77a773faa0bc'],
'0eac306f-7b21-534f-b7eb-2fe3dbf0dcb9':['a255ad13-8d70-5d14-9431-079d5167750e', '95a58411-5323-59d9-ba02-0e746f8ea16e', 'c2ad7853-588c-5dcf-ad06-a295173586eb', '15fd56d7-eaf3-572b-b11d-ea4a8c40dd7c', '8f96bdea-26b6-5304-a068-86bc974e0cff', '78b205cc-391a-5f9c-adfd-289a5027fead'],
'6d6a0abb-b4fa-5dfa-b44e-35927bff8864':['c474b779-c670-590f-b3e6-f3c48a87ff72', 'a4c75988-d2e2-5aef-ac41-20137a78de9d', '3d6393a2-0c9a-5886-b5ed-0dfe331bb9bc', '057fe70d-3f02-5132-a580-50314a6da51e', '91b185f6-5c35-55dd-9c43-1f5789f4693f', '448b7798-a147-5c15-8052-78843751d0ea'],
'3f02111a-3779-53cb-b556-ef2b02712eb8':['5df1139a-fd1f-5e47-ba24-ba6d46f29c98', '2a725ab6-c3a6-55f4-b3b5-61197a6fb7f4', '905898bd-b3a7-59e8-81b9-35bdbd682f5a', 'a265e9b0-1722-5826-ae35-dd64cecc8cd3', '63c9b56f-726f-539d-a347-cbcbcabc7f3a'],
'a2ec645e-4b24-5995-ac16-a91d2fe427d8':['6e59ae8b-a73e-5819-abde-b6275f9afd44', '35615ff3-9c28-5f4d-96d4-75ee7e861ab4', '9806a724-a62a-53f3-bba4-481f40fbecd6', '893f7929-8f1e-5472-a209-503b6fb1b028', 'fb7f54de-114b-56e9-9fc5-6c3011e104ab', '491918a7-5259-5a5e-b01f-4842a98ec78d', '0c04c0a9-fa85-548d-bd89-35300e16b28c', '16584e89-1b38-5bbb-ba45-a7bfc6dcc8e8'],
'c2ad7853-588c-5dcf-ad06-a295173586eb':['6819ac99-458c-5922-8861-14204f4ba663', '7e14c6f9-7e3f-5070-a550-d278ce7d77cc', '7d1e3f8e-a63c-551e-918d-94a09f2a94b1', '102771ca-59e9-5017-8061-54f4d6c79b3e', '01e5edf1-5304-55f7-84f7-570fe39199f8', '262e95ff-8274-5b39-b0f8-2e5c8d980220'],
'7a93383a-3f64-5d88-b0ca-f25210a26530':['19f0adde-24bb-59d3-a21f-a6583714f9a1', 'ab0a0b11-5984-55e5-9ef8-33b003577a0a', '93d033f8-4541-575b-96f8-fb3366b3eb77', 'ff9107a2-0017-57bc-b2aa-8e8880d11598', 'fff722a2-0908-5198-9a69-64612d28f92e', '0b314379-45ee-528d-804d-ef7478198af6', '21298d2b-ce8b-5aef-ba4b-25399928c53f', 'cf1fb9ed-b7a9-5aa0-acbd-69505ead7ce2', '0008f3ef-65a1-5852-91ca-e2de3c8cea7e', 'cfcd8e1d-ec5a-55ae-a82f-8e18f9c2e7ec'],
'8b881676-4524-54fb-8d04-e9d80a70adc6':['6cde2a53-9dc2-5790-b9d9-3e98d685f9b0', '03a57c6b-eef1-5407-a376-9dbc57722c20', '42fd29d1-ec85-561a-94d7-0b010dfba9f7', '4dfb3296-c5ab-5d27-bcaa-71a67990dc0f', '8822903e-754e-577d-9785-fed317439051', '29f2b914-f655-5a0e-8642-252840119f94', '5db27606-2ba6-5599-aab1-754449cbf30f'],
'5e6d242e-3a76-5145-9c3d-9cde3e69fd41':['7e8b428e-dee1-5a3e-a022-afd5940893b3', '91e45209-0c47-51de-b12a-5b8d516b7a2b'],
'44aba920-f4af-547e-84e6-68db5b56617a':['f599c68c-b3be-5beb-ae2c-68538805afe7', 'a196b19e-0313-5588-a132-1a920ff3c9cd', '0d002ee3-3822-5835-b49c-ae18bf616991', '97fdace6-f059-57b6-8e59-5d39e33364ca', '0c8790f8-e268-50c0-8861-e7103fda0db4'],
'1ed96474-1404-5a0e-b5b5-34f1c0638989':['6129066c-1a86-5960-8be5-1d79e52024cb', 'a674ad5e-2de2-522a-b99b-c21b9a815ccb', 'b1a34bd8-63b5-52c3-86c3-3c83409b96bf', '9155ea42-371d-5bc7-950b-5c93e151b4be', 'f5f4427a-f883-5ad7-acfe-adb68e84067c'],
'87bf6c8a-3ca4-5f9d-b552-8d9f857ebd74':['3cfe2a85-be8e-5a29-90ec-df7568441529', '7878a4dd-803d-597c-9773-038931bb341d', '67050e4c-7771-5a63-b48f-8f699f6fd53b', '5ced6ecb-26fd-50a8-b16e-f9844203b854', 'bf155ea4-adda-50ea-aec5-ff3bd4b493e4', '6cfe0483-aacf-56b6-baec-0a30757b4888', '88197a8b-8cc4-56b8-bac1-7c0e92826ce6', '606d25b9-82c9-5862-b5bb-6106b7d649ef', 'a920a145-b5a5-52e3-9137-5468466d87b7'],
'1c599f2e-4e28-5694-9647-0d06f5896a88':['05bb9448-713e-547d-b59c-8a4464e98702', '22c47d21-3704-58a1-bb7d-a3bf98cc766e', 'd9a65fea-a863-578d-81d4-577a9b4f7a03', '98ad153e-e53f-5337-b21a-ee16ebb15ce7'],
'2a07b0ed-7a52-5641-9e9a-e15e04bd2faa':['2500dc34-48b5-53eb-9d15-3ac2431fe1a3', 'c89aa997-e62c-53c5-a3ae-e657297d208a', 'c5b2ae28-843a-5701-872e-c182df1e64ce', 'f57b274a-1598-5e79-b8bf-db8690fd3f50', '363885b4-e937-529e-b9eb-b10989879494', 'f995b9fe-fb7c-5b0f-9673-30203b4e9c1a'],
'27a20afb-730c-5243-bae9-21de6ae9a95d':['fdcb3a4c-1db3-597c-b885-093937c8a68c', '6614e817-c2ae-56ba-8419-5965ab8600ed', '98ac98f5-d676-5eee-9602-c874f6d86f15', 'e52681c3-ac43-5d84-821e-0a8f2323f49f', '3de135e6-0263-5d65-97a9-0b1ffc473c6d'],
'4294552e-cc06-5002-a418-5c295e89f1d7':['7d128a0a-7a33-5ec7-994e-be6b5b44d58a', '78d39ee5-22a5-570e-9103-d918b64296e8', 'ea7463ff-afb9-5e57-b63f-ed4956ab96a5', '535386bd-b0e0-541a-ba30-c4579db6f8aa', 'e140379d-82dc-5d26-b182-3ff7f77f03b2'],
'65c843d5-e8b8-537f-a6c4-3ccb301d09e5':['6deb9c74-e7ed-551a-b88f-a107678b22c1', 'dad69934-838a-5abb-99cc-0661feb0b95d', 'aa0ef5d1-7a0e-518d-b93d-1834db5bae48', 'ddc78420-8f1e-5656-a2fa-c967f278aad1', 'cdd63ae9-1a94-5a88-ad44-4c68105d51fa', 'ba3be4f0-592d-5d72-a96b-5c461c871c9b', '005d3b86-14de-58a3-86ac-16ceff561782', '71b50e66-cabc-5697-8e58-2c43648932db', 'eb6a1519-3559-5797-ac35-d31505d66961', '01289fb6-5c85-50ba-a8b9-bfa754d5aeb5'],
'65418e9d-3d93-505c-94f5-df49bfa49847':['89d4bd99-1a51-568a-bd62-668c227a3595', '26e37b90-e471-5595-abed-bbe8d69e5f1e', '6720b5a6-7bb0-53fa-a8ad-96d9ab01ffe8', 'eae0a255-8263-5ce2-a9d2-a647b9d9273e', '5e906b7d-5d87-587a-a1da-8daba92d28c4'],
'280670b2-7d06-535c-80b7-4438a537a3fc':['32ad8fca-24f6-5b33-b2fa-76f1956f52ad', '1449612a-5f9b-5960-8114-2da760f63c5e', '0d2b68da-87c5-5ef2-89cf-269e00c55ad0', '158cea75-7d77-5f25-b54b-8a5c5fa808fe'],
'b478e741-bb5c-5f1e-ac5c-e58ea15f2358':['3ff2338c-3155-5eaa-a5fa-daa4eaac8b35', '4a9a10a4-45db-5bc9-b79b-715e55109195', 'cc57ea78-01eb-555a-82fd-556d731b6a00', '35aebfd6-8e1a-5cfd-a2e9-2fb75a231ce9', 'c41905d1-28e9-5510-a8ed-e547ac81a141', '236291ef-da5c-5d7e-9aa1-b05d0170d10f', '82c2627c-9d57-591b-b149-bf9c4db05162'],
'87ef73f2-887a-52b0-aa4f-2e1b8d3676cb':['7f1e68e8-3cfb-521a-aa83-aa38c8b8714f', 'a18ecb25-b898-5b03-907a-04067695f9b1', 'fdcb3a4c-1db3-597c-b885-093937c8a68c', 'c918d16d-8bf9-5ae1-a32d-7d0cf78245f0', '9f555599-a017-57f2-8e42-a3eaa845c1f9', '4c4827c7-7501-5c87-a954-b2e186b455d7', '47bd74b8-f707-5094-83dd-fe2e8f0e0dcb', '98ac98f5-d676-5eee-9602-c874f6d86f15'],
'ff46835d-4bb0-592b-9c5f-470b9bdb9d76':['4c0daf64-b423-56db-8f1b-c83344b6f683', '6614e817-c2ae-56ba-8419-5965ab8600ed', '6bfc31d2-afca-56b3-a5f4-d9f19e628c55'],
'e5e55e0c-98f2-57e6-b252-2acd32197a33':['c5cfd93b-0b81-575a-bc65-952d2d6d68fc', '4003165a-47fa-5b8b-9430-d553e3969130', '7689e398-0970-5c18-ba93-e0b14013ab30', '48fa5894-3be0-5046-b99b-96d65d13fbc7'],
'311b72c3-b538-5a62-b2c8-e340ae014465':['aec24c7c-c835-5212-ac86-09f120b762e6', 'e03cdac1-b51d-5de7-bd1f-81d8d84b6d26', '8e417d85-109c-55f2-adc9-f45557b23eaf', '1ff6f9ec-3eb9-547f-be97-4e8ec82a1bc7'],
'fb7f54de-114b-56e9-9fc5-6c3011e104ab':['e4a6a41f-4737-545b-8e8b-938be76fa906', '876fd01f-2096-5772-bd06-dc91ffc165d7', '491918a7-5259-5a5e-b01f-4842a98ec78d'],
'35615ff3-9c28-5f4d-96d4-75ee7e861ab4':['c9004e51-ac87-58aa-8404-4f0543bdab21', 'a2ec645e-4b24-5995-ac16-a91d2fe427d8', '893f7929-8f1e-5472-a209-503b6fb1b028', '3566443e-27c9-5f5b-8627-905d548c484d', '1daea858-e9e3-56ee-bf3e-5ec7a2229d31'],
'fc7226d1-910a-5010-bdb9-2d7865355965':['7b961b85-ed5e-5c58-adc4-74e2cd714399', '705b29f0-5d25-5da9-83e6-1110113eeb69', '6a99e295-db08-5598-9746-10848bd2871e', 'fd7e1ab7-311d-5f0b-ac20-35aa8050659e', 'b5702e82-21e5-5a8f-b0a2-1d349911cc15', 'd45935d0-ac75-510a-b15c-56b4845e4a9e', 'd3189f4b-7dd2-5751-a8d2-a2121b0b4790', 'c89752eb-ebd8-5c4b-b264-b8aa49b25a4c', '62988cd5-c570-5a40-afb2-978322885cbf'],
'70b69899-4663-5c84-9a18-e29f38661aea':['fc7226d1-910a-5010-bdb9-2d7865355965', '705b29f0-5d25-5da9-83e6-1110113eeb69', 'c89aa997-e62c-53c5-a3ae-e657297d208a', 'fd7e1ab7-311d-5f0b-ac20-35aa8050659e', 'ac604aa7-1cfc-56b0-976d-0faafd49e711', 'd3189f4b-7dd2-5751-a8d2-a2121b0b4790', '93dff7a6-8bec-5779-9fa7-d31aa2974689', 'c89752eb-ebd8-5c4b-b264-b8aa49b25a4c'],
}

print("number of public company labels: %d" % sum([len(x) for x in public_labels.values()]))

private_labels = {'08884f97-0714-57a9-a356-fc0fb67dd4ac': ['89f421e8-0aa0-5a71-b8c2-9880b83844d9', '611e5303-4308-5924-9a8a-152af993f001', '3a011750-a6e8-511b-b899-f77a4e6d596e', '781fbc7e-9690-55c8-b179-2bb7cded8cf9'],
                  '5b28ef54-facf-5f87-b37b-d2cee2cdd466': ['6715d0e1-a229-5de7-b6e3-046a8e103e55'],
                  '39b70d92-2958-5ecf-abcb-a772b565afae': ['a59853ba-5513-5b03-baae-c41c82eda078', 'a8658765-5012-5bac-a052-39e1ada83bd7', '44c749dd-2f92-56b9-831d-95450655a245', 'c20fa5cc-7a19-569a-80d0-6dcc8579cb8c'],
                  'a0b22be6-dd87-5e6a-af12-f967ed7f481d': ['b562fb43-42a9-5eea-aa30-0d6c5812dc58', '74fc7730-afff-5593-aa11-ad68af06cd36', '0df40d87-a948-5a65-9a02-0487173aa75a', '53cfd17f-fe28-51e0-8db6-a334e43496cf', 'b3227a8e-b565-5a27-9dcf-b0f3ff4c1ae5'],
                  '25e9ea1b-3e45-5503-bc84-cdc0b691e8f7': ['56086072-9a3e-5bce-8a30-bd0bbfc8e316',
                                                           '63d71571-f43e-56fd-88f0-423fed6fa72a',
                                                           'fed07c4d-bb05-5d15-9d1b-d99e3fdbc588',
                                                           '0cce8c17-86af-581c-be18-cfea9fbecee8',
                                                           '6175abaa-401f-59a9-874e-71205592a62e',
                                                           '1993f84b-73f6-5547-a56a-d23987a357c5',
                                                           'a367d8ee-c870-53be-9f06-9e68feebcd4e'],
                  'd62478ba-4467-58fb-a741-8aadf82f73ca': ['f42a2e04-c602-5884-bdc6-63bc14ae5ea2',
                                                           'c66f15aa-4626-5793-a41d-e25f708e35c1',
                                                           'cc86ec41-7df7-5912-89c4-05dd57b5f12b',
                                                           '1f729853-87f8-522b-8be3-79a5def6e551',
                                                           '67e34d98-f316-531e-9307-f176e1aa7c0b',
                                                           '1123a383-aa82-5f88-8dbd-3494275e4a56',
                                                           '86f77c78-d551-5185-951a-0e943f69b469'],
                  '9f9fdf8d-bb27-5a1a-a20a-850b54799ee1': ['0ea0fa26-7ca8-5257-b6c7-b8ba8e2bd5a8',
                                                           'e303f46e-40ab-5d8f-9aab-7ff14b77a0b2',
                                                           'a63cdfc8-2e64-57da-a362-bb5f14b05ef0',
                                                           'c543aa03-aa0b-5675-9602-7803004eb729'],
                  '2a253eb1-9353-544a-a9a9-a1ad76a99321': ['dbfbfed4-53d5-581e-a9ed-8b2c9163d484',
                                                           '4253351d-4923-5958-a860-bf8ba9b358b6',
                                                           'e0c78a14-c9cd-580f-bbb8-164b894a3f69',
                                                           '6103d90a-98d4-5cb5-bbee-90b11df95c4b'],
                  'a74ed87e-0a73-5c61-b71a-da164a2fe263': ['e672a67e-3b7a-5848-9847-b2bbdf21a2b7',
                                                           'ce6aeec8-2493-58c3-84a2-f19b1ee4f29d',
                                                           'b08e4067-8995-5d08-9dd7-5951a00ba857',
                                                           'cd20d61a-5f62-5e41-8dc8-46afeff0c6f8'],
                  '396ab62b-bc96-5161-9389-88531acd6db9': ['85b784a3-e117-5991-9cb0-ee6c8834d897',
                                                           '5dfa9eae-ef27-5640-9cfc-652965cd1835',
                                                           'bddc2947-e5de-5db8-878a-9b0657d43716',
                                                           'f8133b15-315c-5178-b5ea-c38a18ed7746',
                                                           'eb175ee3-f440-5459-96e3-72e800efc11c'],
                  '7a69b5e2-043f-5d72-bd30-46f7de988db0': ['dd21320b-ec32-5c28-9600-52bbbc0b2eb9',
                                                           'caa0de57-5580-5189-836f-c9769aaca726',
                                                           'a15ae251-9e6b-58b4-8cb4-b0aad975a6fe',
                                                           '3f630085-fdc0-591a-a1bf-86256334225a',
                                                           'c2c42990-2b1c-575d-ac49-f91ae9751313'],
                  'a154dd44-38f8-54b1-b1c0-8fa6b7d14e7f': ['90c20cd1-0d40-5f26-8e77-8f6202e6eb3a',
                                                           '5302c6dc-cc1d-5774-93e0-f43345d45dd0',
                                                           '6d360e9a-c698-5301-b92e-9ca9eecaefe3',
                                                           '11b55d94-55c1-5f77-b94b-0bd5b3cc90f1'],
                  }

negative_labels = {'95b851bc-621e-5351-96f8-75d4e4828534': ['07bb2fa4-504e-596c-8cc1-48131bcb683e'],
                   '30e94db2-4762-58bd-83ec-fd866ab80a90': ['a5209416-58e3-5774-8668-7fb02ea9828b', 'c60a23f3-5490-58c4-abfd-06d1c3047853', '2e63211a-1804-5956-899f-49b8bffe3dd8', 'b1617dbb-d718-55f6-ade5-744c37e677be', 'be98a208-061e-5317-9c71-97e3c9a73825'],
                   'df671f51-4e93-5c03-ad4b-4efcb5e028ed': ['f35a6457-9496-5e4c-a40a-9b70c43f3d20', '396201b2-c6c9-585e-adbc-aa88968f32fb', '2dc9957b-a41b-5c3d-af5b-70debd1b2a15', '6649b3e5-73ad-51c1-921e-63a8d7618ca5', 'd48441e2-a1d1-5771-b8bc-d79aa880546f', '97bc423e-4088-51f9-8a25-8a546457431c'],
                   'b3dd3855-58c8-53d0-8e39-107b911575bb': ['5d85f070-67a2-57b6-b753-69732701539d', '0a4ac21f-bee8-59e9-8939-a0a4d820839f', 'a31dae50-6ff3-5a27-ac9e-160c1fd65221'],
                   }


HOST = "host.docker.internal"
PORT = "4443"
SSL = True
INDEX = "company"

if SSL:
    ssl_context = create_ssl_context()
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE
else:
    ssl_context = None

client = Elasticsearch(
    hosts=[{'host': HOST, 'port': PORT}],
    indices=[INDEX],
    scheme="https" if SSL else "http",
    ssl_context=ssl_context)

ranker = Ranker()

number of public company labels: 215


In [53]:
def obtain_result(doc_id,
                  min_doc_freq=1, 
                  min_term_freq=0, 
                  max_query_terms=15,
                  mlt_field="business_description"):
    try:
        query_doc = client.get(index='company', id=doc_id)
    except NotFoundError:
        return []

    query = {"query":{'bool': {'must': []}}}
    
    company_revenue = query_doc["_source"].get("total_revenue")
    company_ebitda = query_doc["_source"].get("ebitda")
    company_region = query_doc["_source"].get("region")

    fields_based_filter: dict = defaultdict(list)

    # If the query is empty, include a default filters
    if company_revenue is not None:
        min_revenue = company_revenue / 10.0
        max_revenue = company_revenue * 10.0
        fields_based_filter['must'].extend(
            [
                {'range': {'total_revenue': {'gt': min_revenue, 'lt': max_revenue}}},
            ]
        )

    if company_region is not None:
        fields_based_filter['should'].append({"match": {"region": company_region}})

    query["query"]["bool"] = dict(fields_based_filter)

    mlt = {
        'more_like_this': {
            "like": [{"_id": doc_id, "_index": INDEX}],
            "fields": [mlt_field, 'primary_sic_node_desc'],
            "min_doc_freq": min_doc_freq,
            "min_term_freq": min_term_freq,
            "max_query_terms": max_query_terms,
            "include": True,
        }
    }

    query_boolean_should = query["query"]["bool"].get("should", [])
    query_boolean_should.append(mlt)
    query["query"]["bool"]["should"] = query_boolean_should
    query["size"] = 100

    json_result = client.search(index=INDEX, body=query)
    
    results = json_result["hits"]["hits"]
    
    ret = []
    
    for result in results:
        src = result["_source"]
        ret.append(
            Company(
                id=src["entity_id"],
                name=src["name"],
                revenue=src.get("revenue"),
                ebitda=src.get("ebitda"),
                ebitda_marg=src.get("ebitda_marg"),
                sector=src.get("primary_sic_node_desc"),
                num_employees=src.get("number_employees"),
                country=src.get("country_of_incorporation"),
                score=src.get(result["_score"]),
                region=src.get("region"),
                description=src.get(mlt_field),
                company_type=src.get("company_type_name"),
            )
        )
            
    return ret


def obtain_result_old(doc_id, 
                  min_doc_freq=1, 
                  min_term_freq=0, 
                  max_query_terms=15,
                  mlt_field="business_description"
                 ):
    
    try:
        query = client.get(index='company', id=doc_id)
    except NotFoundError:
        return []

    revenue_range = 0.1
    try:
        revenue = query['_source']['total_revenue']
    except:
        print("Missing revenue: " + doc_id)
        return []
    revenue_lower_bound = revenue * revenue_range
    revenue_upper_bound = revenue / revenue_range
    
    print(revenue_lower_bound, revenue_upper_bound)

    filter_queries = [  # Q('match_phrase', company_type=query['_source']['company_type']),
        Q('range', total_revenue={'gt': revenue_lower_bound, 'lt': revenue_upper_bound}),
        ]#Q('range', ebitda_marg={'gt': 0})]

    region = ""  # query['_source']['region']
    keywords = ""

    if region:
        filter_queries.append(Q('match', region=region))

    if keywords:
        filter_queries.append(Q('match', business_description=keywords))

    q = Q('bool',
          filter=filter_queries,
          should=[Q('more_like_this',
                    like=[{"_id": doc_id, "_index": INDEX}],
                    fields=[
                        "primary_sic_node_desc", #sic_code_desc #industry_class_code_desc
                        mlt_field,
                    ],
                    min_doc_freq=min_doc_freq,
                    min_term_freq=min_term_freq,
                    max_query_terms=max_query_terms,
                    include=True, )]
          )
    
    s = (
        Search(index='company')
            .using(client)
            .query(q)
            # .extra(explain=True)
            .params(search_type="dfs_query_then_fetch")
    )

    max_score = -1
    results = []

    for hit in s[0:100]:
        id = hit.meta.id

        score = hit.meta.score
        max_score = max(max_score, score)

        cname = hit.name

        sector = (
            hit.primary_sic_node
            if hasattr(hit, "primary_sic_node")
            else ""
        )
        country = hit.country_of_incorporation if hasattr(hit, "country_of_incorporation") else ""
        company_type = hit.company_type if hasattr(hit, "company_type") else ""

        revenue = float(hit.total_revenue) if hasattr(hit, "total_revenue") else 0
        num_employees = (
            hit.number_of_employees if hasattr(hit, "number_of_employees") else 0
        )

        region = hit.region if hasattr(hit, "region") else "n/a"

        description = hit.business_description if hasattr(hit, "business_description") else ""

        ebitda = float(hit.ebitda) if hasattr(hit, "ebitda") else 0

        ebitda_marg = ebitda/revenue if revenue != 0 else 0

        results.append(
            Company(
                id=id,
                name=cname,
                revenue=revenue,
                ebitda=ebitda,
                ebitda_marg=ebitda_marg,
                sector=sector,
                num_employees=num_employees,
                country=country,
                score=score,
                region=region,
                description=description,
                company_type=company_type,
            )
        )
    
    results = ranker.rank_by_revenue(query, results) + results[21:]
    return results


def find_peer_in_result(results, peer):
    for i, company in enumerate(results):
        if peer == company.id:
            return i

    return -1


def compute_counts_from_ranks(ranks):
    counts = [0] * 6
    for rank in ranks:
        if 0 < rank <= 10:
            counts[0] += 1
        elif 10 < rank <= 20:
            counts[1] += 1
        elif 20 < rank <= 50:
            counts[2] += 1
        elif 50 < rank <= 100:
            counts[3] += 1
        elif 100 < rank:
            counts[4] += 1
        else:
            counts[5] += 1
    return counts

In [54]:
# obtain_result("1b3872ba-8913-517f-a253-7e62a525cf55")
# obtain_result_old("1b3872ba-8913-517f-a253-7e62a525cf55")

In [55]:
top10_ratio = []
top20_ratio = []

# for min_doc_freq in [0,1,2,3]:
#     for min_term_freq in [0,1,2]:
#         for max_query_terms in [5, 7, 10, 12, 15]:

ranks_default = []
ranks_model = []
ranks_model_soted = []

doub_avg_top10 = []
doub_avg_top20 = []

missing_case = {}

negative_count = 0

suggested_peers = {}
for query_id, public_label in public_labels.items():
    tmp_ranks_default = []
    tmp_ranks_model = []
    tmp_ranks_sorted = []

    missing_case[query_id] = []

    # results = obtain_result(query_id, min_doc_freq, min_term_freq, max_query_terms)
    results = obtain_result(query_id, 1, 0, 10)
    
    if len(results) == 0:
        print(f"Failed to find target company {query_id}")

    suggested_peers[query_id] = results[:20]
    
    for peer in public_label[0:]:
        ranks_default.append(find_peer_in_result(results, peer))

        tmp_ranks_default.append(find_peer_in_result(results, peer))

        if find_peer_in_result(results, peer) == -1:
            missing_case[query_id].append(peer)

    # print(tmp_ranks_default)

    if query_id in negative_labels:
        for negative_label in negative_labels[query_id]:
            if find_peer_in_result(results, negative_label) != -1:
                negative_count += 1
                # print(negative_label)

    doub_avg_top10.append(len([x for x in tmp_ranks_default if 0 < x <= 10]) / (len(tmp_ranks_default)))
    doub_avg_top20.append(len([x for x in tmp_ranks_default if 0 < x <= 20]) / (len(tmp_ranks_default)))

    print(tmp_ranks_default)

print(compute_counts_from_ranks(ranks_default))
print("MRR: %f" % np.mean([1 / rank for rank in ranks_default if 0 < rank <= 20]))
print("Number of base companies: %d" % len(doub_avg_top10))
print("Mean top 10: %f" % np.mean(doub_avg_top10))
print("Mean top 20: %f" % np.mean(doub_avg_top20))
print("Negative count: %d" % negative_count)
print()

top10_ratio.append(np.mean(doub_avg_top10))
top20_ratio.append(np.mean(doub_avg_top20))

print(top10_ratio)
print(top20_ratio)

[77, 6, 4, 12]
[-1, -1, -1]
[-1, -1, -1]
[23, 11]
[68, 57, 90]
[18, 11, -1]
[-1, -1, -1, 14, 7, -1, -1, -1, 22, 10]
[10, 9, 1]
[23, 47, 4, 61, 2, 6, 98]
[5, 13, 7, 1, 20]
[-1, -1]
[46, 31, -1, -1, -1, -1]
[-1, 19, 9, 17, 18]
[2, -1, -1, -1, -1, -1]
[74, 1, -1, -1, -1, 66]
[-1, 44, -1, -1, -1]
[96, 25, 13, 8, 21, 4, 6, 20]
[6, -1, -1, 22, -1, -1]
[-1, -1, 14, 30, -1, 7, 5, 12, -1, 28]
[-1, -1, 53, -1, -1, -1, -1]
[-1, -1]
[99, -1, 66, 4, -1]
[-1, 5, 22, 10, -1]
[33, -1, -1, -1, -1, -1, -1, 3, -1]
[-1, -1, -1, -1]
[2, -1, 14, 21, -1, 78]
[49, 13, -1, 15, 3]
[-1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, 70, -1, -1, -1, -1]
[-1, -1, -1, -1, -1]
[-1, 17, -1, -1]
[-1, -1, -1, -1, -1, 95, 39]
[-1, 30, -1, -1, -1, -1, -1, -1]
[4, 26, 5]
[-1, 1, -1, -1]
[-1, 15, 32, 12]
[13, 6, 14]
[-1, 59, 15, 9, 77]
[22, 5, 18, 25, -1, 10, 14, 12, 4]
[29, 73, 33, 28, 91, 6, 9, 76]
[38, 26, 25, 20, 0, 106]
MRR: 0.201275
Number of base companies: 40
Mean top 10: 0.177867
Mean top 20: 0.314950
Negative count: 0

[0.

In [ ]:
for target, peers in missing_case.items():
    

In [4]:
def company_to_dict(c):
    return {
        "id": c.id,
        "name": c.name,
        "country": c.country,
        "sector": c.sector,
        "revenue": c.revenue,
        "ebitda": c.ebitda,
        "num_employees": c.num_employees,
        "score": c.score,
        "region": c.region,
        "description": c.description,
        "company_type": c.company_type,
    }

In [5]:
import smart_open
import pickle

In [10]:
with smart_open.open("s3://onai-ml-dev-eu-west-1/company2vec/model/es_mlt/suggested_peers.pkl", "wb") as f:
    pickle.dump({k:[company_to_dict(el) for el in v] for k,v in suggested_peers.items()}, f)